In [ ]:
from brian2 import *
import matplotlib.pyplot as plt

start_scope()

# Parámetros del "Módulo Cortical Universal"
n = 100
tau = 20*ms
v_rest = -70*mV
v_threshold = -50*mV

# Modelo de neurona con adaptación (para que sea flexible)
eqs = '''
dv/dt = (v_rest - v + I_syn + I_ext) / tau : volt
dI_syn/dt = -I_syn/tau_syn : volt
tau_syn = 5*ms : second
I_ext : volt  # Entrada desde otras áreas
'''

Corteza = NeuronGroup(n, eqs, threshold='v > v_threshold', reset='v = -85*mV', method='exact')
Corteza.v = v_rest

# Conexiones internas (Recurrencia: fundamental en el neocórtex)
S_int = Synapses(Corteza, Corteza, on_pre='I_syn += 0.5*mV')
S_int.connect(p=0.1)

# Monitor
m = SpikeMonitor(Corteza)

# FASE 1: Recibimos una entrada lenta (Baja frecuencia)
Corteza.I_ext = 15*mV
run(200*ms)

# FASE 2: Silencio
Corteza.I_ext = 0*mV
run(100*ms)

# FASE 3: Recibimos una entrada fuerte (Alta frecuencia/Urgencia)
Corteza.I_ext = 40*mV
run(200*ms)

# Visualización
plt.figure(figsize=(10, 5))
plt.plot(m.t/ms, m.i, '.k', markersize=2)
plt.axvspan(0, 200, color='blue', alpha=0.1, label='Entrada Débil')
plt.axvspan(300, 500, color='red', alpha=0.1, label='Entrada Fuerte')
plt.title('Módulo Cortical: Misma estructura, diferente respuesta')
plt.xlabel('Tiempo (ms)')
plt.ylabel('Neurona index')
plt.legend()
plt.show()